In [8]:
import pandas as pd
import numpy as np


from core import constants
from core.utils import *

log = get_logger()

## RPGF 3 Data Check and Cleanup

In [9]:
df = pd.read_csv("data/dummy_data_rpgf3.csv")

In [10]:
df.head()

,Address,Has voted,Has published,Published at,Created at,Updated at,Projects in ballot,Votes
0,5R3d7IeoAB,True,False,NaN,2023-11-06 10:51:07,2023-11-13 09:28:20,NaN,[]
1,K66AqbQbXq,False,False,NaN,2023-10-30 22:32:19,2023-11-14 00:08:19,NaN,[]
2,l6UWM6Bf4X,True,True,2023-11-02 18:36:08,2023-10-24 21:11:07,2023-11-08 13:11:34,240.0,"[{'amount': '644777', 'projectId': 'proj274'},..."
3,vWgy70oDLH,False,False,NaN,2023-09-05 00:10:01,2023-10-21 02:55:52,NaN,[]
4,JEkkCBV3nk,True,False,NaN,2023-09-19 13:30:58,2023-09-30 02:28:27,NaN,[]


In [11]:
# Check if voter_address is unique
if df["Address"].nunique() == df.shape[0]:
    log.info("Address is unique.")
else:
    diff = df.shape[0] - df["Address"].nunique()
    log.info(f"Address is not unique. There are {diff} duplicates.")

# Check if all voters have voted
if df[df["Has voted"] == False].shape[0] > 0:
    not_voted = df[df["Has voted"] == False].shape[0]
    total = df["Address"].nunique()
    log.info(f"{not_voted} voters out of {total} have not voted.")
else:
    log.info("All voters have voted.")

2023-11-29 15:33:51 INFO | Address is unique.
2023-11-29 15:33:51 INFO | 518 voters out of 1000 have not voted.


In [12]:
# Apply the function and concatenate results
expanded_list = [
    expand_json(safe_json_loads(row), idx) for idx, row in df["Votes"].items()
]
expanded_df = pd.concat(expanded_list, ignore_index=True)

result_df = expanded_df.set_index("original_index").join(df.set_index(df.index))

TypeError: the JSON object must be str, bytes or bytearray, not list

In [ ]:
columns = [col for col in result_df.columns if col not in ["amount", "projectId"]]
columns += ["amount", "projectId"]  # Add the columns to the end of the list
result_df = result_df[columns]

# Update df columns names
result_df.columns = [
    "voter_address",
    "has_voted",
    "has_published",
    "published_at",
    "created_at",
    "updated_at",
    "projects_in_ballot",
    "votes",
    "amount",
    "project_id",
]

result_df.drop(columns="votes", inplace=True)

In [ ]:
result_df.head()

## Calculate Voting Results

In [ ]:
allocator = ProjectAllocator(
    total_amount=constants.TOTAL_AMOUNT,
    min_amount=constants.MIN_AMOUNT,
    quorum=constants.QUORUM,
)

In [ ]:
# Calculate if a project meets the quorum and calculate the initial allocation based on median
initial_allocation = allocator.calculate_initial_allocation(result_df)

In [ ]:
# Scaling the total to 30M OP by project and filter out those with < 1500 OP
allocation_iter = initial_allocation.copy()
allocation_iter["scaled_amount"] = allocation_iter["median_amount"]

# Set a maximum number of iterations to prevent infinite loop
max_iterations = 10
current_iteration = 0

while (
    allocation_iter["scaled_amount"].sum() != constants.TOTAL_AMOUNT
    and current_iteration < max_iterations
):
    allocation_iter = allocator.scale_allocations(allocation_iter)
    current_iteration += 1

In [ ]:
allocation_iter

In [ ]:
# Check if the loop exited due to reaching max iterations
if (
    current_iteration == max_iterations
    and allocation_iter["scaled_amount"].sum() != constants.TOTAL_AMOUNT
):
    log.info("Maximum iterations reached without meeting the total amount condition.")
else:
    final_total = allocation_iter["scaled_amount"].sum()
    log.info(
        f"Condition met with {final_total} OP allocated through {current_iteration} iteration(s)."
    )

In [ ]:
# export csv
# allocation_iter.drop(columns="median_amount", inplace=True)
allocation_iter.to_csv("data/rpgf3_allocation_final.csv")

log.info(f"Results saved in data/rpgf3_allocation_final.csv.")